In [45]:
import pandas as pd
import numpy as np
import pycaret


census_pth = "data/census.csv"
data = pd.read_csv(census_pth)
cols = data.columns.to_list()

In [46]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
import pandas as pd

#get the data from the dir and save it as a pandas' dataframe
def get_data():
    dataset = pd.read_csv("data/census.csv")
    X = pd.DataFrame(dataset)
    cleaned_column_names = (X.columns
                            .str.strip()
                            .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
                            .str.lower()
                            .str.replace('[ _-]+', '_')
                            .str.replace('[}{)(><.!?\\\\:;,-]', ''))
    X.columns = cleaned_column_names
    # group the incomplete high school education into a bin called some-HS
    someHS = ['9th', '10th', '11th', '12th']
    for i in someHS:
        X["education"]= X["education"].str.replace(i,'Some-HS')
    return X



In [47]:
def process_data(
    X, categorical_features=[], label=None, training=True, encoder=None, lb=None
):
    # point to the X output from the get_data as an automate cleaning process that would be 
    # benefical for both train and test data.
    try: 
        isinstance(X, object)== isinstance(X.dtypes,(np.int64,object))
        #return X
    except AssertionError as err:
        raise err
    label = 'salary'
    if label is not None:
        y = X[label]
        X = X.drop([label], axis=1)
    else:
        y = np.array([])

    # get the categorical columns to a list
    for cat in X.select_dtypes(['object']).columns:
        X[cat] = X[cat].str.strip()

        """replace the columns with '?' which are all in the categorical columns with 'unknown'
        keeping this rows because there is not more than 2 values with '? per row, 
        and believe rows with '?' still has valuable information for the model"""

        X[cat] = X[cat].str.replace('?', 'unknown', regex=False)
        categorical_features.append(cat)

    X_categorical = X[categorical_features].values
    X_continuous = X.drop(*[categorical_features], axis=1)

    if training is True:
        encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
        lb = LabelBinarizer()
        X_categorical = encoder.fit_transform(X_categorical)
        y = lb.fit_transform(y.values).ravel()
    else:
        X_categorical = encoder.transform(X_categorical)
        try:
            y = lb.transform(y.values).ravel()
        # Catch the case where y is None because we're doing inference.
        except AttributeError as err:
            raise err

    X = np.concatenate([X_continuous, X_categorical], axis=1)
    return X, y, encoder, lb

In [48]:
x = get_data()
x

,age,workclass,fnlgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,Some-HS,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [52]:
import warnings
warnings.filterwarnings("ignore")# init setup

,Description,Value
0,session_id,7262
1,Target,salary
2,Target Type,Binary
3,Label Encoded,"<=50K: 0, >50K: 1"
4,Original Data,"(32561, 15)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


2022/07/20 22:39:16 INFO mlflow.tracking.fluent: Experiment with name 'Census_Application' does not exist. Creating a new experiment.


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8699,0.9253,0.6611,0.7718,0.7120,0.6287,0.6319,0.1160
gbc,Gradient Boosting Classifier,0.8628,0.9188,0.5873,0.7956,0.6756,0.5911,0.6022,1.2150
ada,Ada Boost Classifier,0.8607,0.9134,0.6165,0.7658,0.6830,0.5951,0.6009,0.3260
rf,Random Forest Classifier,0.8528,0.9012,0.6220,0.7327,0.6727,0.5786,0.5819,0.6870
lda,Linear Discriminant Analysis,0.8397,0.8920,0.5657,0.7160,0.6320,0.5313,0.5374,0.2760
ridge,Ridge Classifier,0.8380,0.0000,0.5111,0.7432,0.6056,0.5080,0.5222,0.0330
et,Extra Trees Classifier,0.8319,0.8747,0.6039,0.6722,0.6361,0.5272,0.5286,0.9760
dt,Decision Tree Classifier,0.8101,0.7420,0.6095,0.6099,0.6096,0.4841,0.4842,0.0870
lr,Logistic Regression,0.7962,0.5868,0.2697,0.7157,0.3916,0.2982,0.3497,0.6730
nb,Naive Bayes,0.7940,0.8033,0.3149,0.6607,0.4265,0.3197,0.3523,0.0400


In [62]:
# check feature importance
plot_model(best, plot = 'feature', save=True)


'Feature Importance.png'

In [64]:
# finalize the model
final_best = finalize_model(best)
# save model to disk
save_model(final_best, './model/census_clf')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='salary',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strat...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, importance_type='split',
                                 learning_rate=0.1, max_depth=-1,
                                 min_child_samples=20,